In [ ]:
#LOADS NECESSARY LIBRARIES TO FETCH AND SUMMARIZE WEBSITE CONTENT
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

#FOR CONNECTING LLaMA3 MODEL LOCALLY USING OLLAMA
from openai import OpenAI
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
#TEST MSG
message = "Hello World"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
#FOR IMIRATING A BROWSER
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
#FETCHES A WEBPAGE  CLEANS BY REMOVING SCRIPTS, IMAGES, ETC.
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
#TESTING BY PRINTING TITLE AND TEXT
w = Website("https://www.grammarly.com/")
print(w.title)
print(w.text)

In [ ]:
#SYSTEM-LEVEL INTRUCTION FOR LLM
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
#USER PROMPT FOR THE MODEL
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(w))

In [ ]:
#EXAMPLE OF DIFFERENT SYSTEM PERSONALITY
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]
response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

In [ ]:
# COMBINES SYSTEM  USER PROMPTS TO SEND TO LLM
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]
messages_for(w)


In [ ]:
#FETCHES THE SITE, CREATES PROMPT AND GET'S A SUMMARY FROM LLM
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

#TESTING
summarize("https://www.grammarly.com/")

In [33]:
#DISPLAYS MARKDOWN SUMMARY
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

#TESTING
display_summary("https://www.grammarly.com/")

### Summary of Grammarly Website

#### Introduction

**Grammarly** is a free AI writing assistant designed to improve everyday communication.

#### Key Features and Benefits

*   **AI Writing Assistance**: Use Grammarly to make text more persuasive, use writing suggestions to add a deadline to a Slack message.
*   **Cross-Platform Compatibility**: Works across all apps and sites you use without copying or pasting content.
*   **Personalized Suggestions**: Get tailored feedback based on what you're writing and who will read it.

#### Features of Grammarly

Grammly offers several AI-powered tools to improve writing quality, including a grammar checker, plagiarism checker and citation generator, paraphrasing tool and more.

**Security**

Grammarly is committed to protecting user data and maintaining confidentiality with no commercialization.

#### Pricing and Plans

Grammarly caters to both individual and enterprise users offering three primary plans: free, personal pro ($0-$983), and team pro ($983-$2499). For larger organizations, Grammarly offers more extensive features such as Data loss prevention and dedicated management.